In [1]:
# Bibliotecas genéricas

import pandas as pd 
import numpy as np
import requests
import time
import json
import os   
import sys
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
from matplotlib import rcParams
from datetime import timedelta
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import concurrent
import plotly.express as px


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Dados de Voos

In [2]:
destino = np.array(['SBKP', 'SBGR', 'SBBR', 'SBCF', 'SBSP', 'SBRJ', 'SBFL', 'SBCT',
       'SBRF', 'SBPA', 'SBSV', 'SBGL'], dtype=object)

#### Dados da ANAC

In [3]:
# Dados de voos de vra 

# SBGR - Guarulhos

# Listar todos os arquivos na pasta 'vra'
vra_files = [os.path.join('vra', f) for f in os.listdir('vra') if f.endswith('.csv')]

# Concatenar todos os arquivos da pasta 'vra' que atendem à condição
vra_list = []
for file in vra_files:
    df = pd.read_csv(file, sep=';')
    df_filtered = df[df['Sigla ICAO Aeroporto Destino'].isin(destino)]
    vra_list.append(df_filtered)

vra = pd.concat(vra_list, ignore_index=True)

# Converter colunas de datas do vra para o padrão internacional ISO 8601 com horário

vra['Partida Real'] = pd.to_datetime(vra['Partida Real'],  format='%d/%m/%Y %H:%M')
vra['Chegada Real'] = pd.to_datetime(vra['Chegada Real'], format='%d/%m/%Y %H:%M')
vra['Partida Prevista'] = pd.to_datetime(vra['Partida Prevista'],  format='%d/%m/%Y %H:%M')
vra['Chegada Prevista'] = pd.to_datetime(vra['Chegada Prevista'],  format='%d/%m/%Y %H:%M')
vra['Horário de Referência para Partida'] = np.where(vra['Partida Real'].isnull(), vra['Partida Prevista'], vra['Partida Real'])
vra['Horário de Referência para Chegada'] = np.where(vra['Chegada Real'].isnull(), vra['Chegada Prevista'], vra['Chegada Real'])

vra.head()

/tmp/ipykernel_968/4125573331.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')
/tmp/ipykernel_968/4125573331.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')
/tmp/ipykernel_968/4125573331.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')
/tmp/ipykernel_968/4125573331.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')
/tmp/ipykernel_968/4125573331.py:16: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';')


,Sigla ICAO Empresa Aérea,Empresa Aérea,Número Voo,Código DI,Código Tipo Linha,Modelo Equipamento,Número de Assentos,Sigla ICAO Aeroporto Origem,Descrição Aeroporto Origem,Partida Prevista,...,Descrição Aeroporto Destino,Chegada Prevista,Chegada Real,Situação Voo,Justificativa,Referência,Situação Partida,Situação Chegada,Horário de Referência para Partida,Horário de Referência para Chegada
0,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5945,6,N,C208,0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2022-11-01 16:46:00,REALIZADO,NaN,01/11/2022 00:00:00,NaN,NaN,2022-11-01 14:48:00,2022-11-01 16:46:00
1,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5122,4,N,C208,9,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2022-11-02 11:58:00,REALIZADO,NaN,02/11/2022 00:00:00,NaN,NaN,2022-11-02 09:45:00,2022-11-02 11:58:00
2,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5950,2,N,C208,9,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2022-11-02 14:19:00,REALIZADO,NaN,02/11/2022 00:00:00,NaN,NaN,2022-11-02 12:24:00,2022-11-02 14:19:00
3,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5960,2,N,C208,0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2022-12-20 11:27:00,REALIZADO,NaN,20/12/2022 00:00:00,NaN,NaN,2022-12-20 09:34:00,2022-12-20 11:27:00
4,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),5927,2,N,C208,0,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,CONGONHAS - SÃO PAULO - SP - BRASIL,NaT,2023-02-20 10:40:00,REALIZADO,NaN,20/02/2023 00:00:00,NaN,NaN,2023-02-20 08:43:00,2023-02-20 10:40:00


### Dados de esperas



In [4]:
# Carregando dados de esperas em voo

esperas = pd.read_csv('esperas.csv')

esperas['hora'] = pd.to_datetime(esperas['hora'], unit='ms')

esperas[esperas['esperas'] > 0 ].head(20)

,esperas,hora,aero
49,1,2022-06-01 01:00:00,SBCT
142,2,2022-06-01 22:00:00,SBGR
153,6,2022-06-01 09:00:00,SBKP
154,3,2022-06-01 10:00:00,SBKP
254,1,2022-06-01 14:00:00,SBSP
260,1,2022-06-01 20:00:00,SBSP
263,1,2022-06-01 23:00:00,SBSP
374,2,2022-06-02 14:00:00,SBCF
385,1,2022-06-01 01:00:00,SBCT
550,2,2022-06-01 22:00:00,SBGR


### Dados Metereológicos
Link para interpretar dados metereológicos: [[link]](https://ajuda.decea.mil.br/base-de-conhecimento/como-decodificar-o-metar-e-o-speci/)

Exemplo: 'METAR SBGL 131000Z 31015G27KT 280V350 4000 1800N R10/P2000 +TSRA FEW005 FEW010CB SCT018 BKN025 10/03 Q0995 REDZ WS R10 W12/H75='
Também é possível uitlizar bibliotecas para interpretar os dados metereológicos, como a `metar` [[link]](https://pypi.org/project/metar/)


Instalando biblioteca para interpretar dados metar: 


In [5]:
!pip install metar

from metar import Metar

Defaulting to user installation because normal site-packages is not writeable


Função para transformar dados


In [6]:
def get_metar_data(x):
    try:
        metar = Metar.Metar(x)
        # Organizar os dados em um dicionário
        data = {
            'station': metar.station_id,
            'time': metar.time,
            'wind_direction': metar.wind_dir.value() if metar.wind_dir else None,
            'wind_speed': metar.wind_speed.value('KT') if metar.wind_speed else None,
            'temperature': metar.temp.value('C') if metar.temp else None,
            'dew_point': metar.dewpt.value('C') if metar.dewpt else None,
            'visibility': metar.vis.value('M') if metar.vis else None,
            'pressure': metar.press.value('MB') if metar.press else None,
            'sky_conditions': metar.sky_conditions() if metar.sky_conditions() else None
        }
        return data # Tenta dividir 10 por y
    except:
        return ''

#### METAF


In [7]:
# Carregando os dados de METAF 


if not os.path.exists('metaf_translated.csv'):
    metaf = pd.read_csv('metaf.csv')

    metaf['hora'] = pd.to_datetime(metaf['hora'], unit='ms')

    metaf['metaf'] = metaf['metaf'].str.replace('METAF', 'METAR')

    # Traduzindo metaf

    metaf['metar_translated'] = metaf['metaf'].apply(lambda x: get_metar_data(x))
    data = metaf['metar_translated']
    new_columns = pd.DataFrame(data.to_dict()).T
    metaf = pd.concat([metaf, new_columns], axis=1)
    metaf.to_csv('metaf_translated.csv', index=False)   
else:
    metaf = pd.read_csv('metaf_translated.csv')
    
metaf.head()

metaf['hora'] = pd.to_datetime(metaf['hora'])



#### METAR


In [8]:

# Verifica se já existe o arquivo metar_translated.csv

if not os.path.exists('metar_translated.csv'):

    # Carregando dados do METAR

    metar = pd.read_csv('metar.csv')

    # Transformando colunas de data

    metar['hora'] = pd.to_datetime(metar['hora'], unit='ms')

    # Traduzindo metar 

    metar['metar_translated'] = metar['metar'].apply(lambda x: get_metar_data(x))
    data = metar['metar_translated']
    new_columns = pd.DataFrame(data.to_dict()).T
    metar = pd.concat([metar, new_columns], axis=1)
    metar.to_csv('metar_translated.csv', index=False)   
else:
    metar = pd.read_csv('metar_translated.csv')
    
metar.head()
    
metar['hora'] = pd.to_datetime(metar['hora'])
    
    


### Juntando Informações

Inserindo informações da ANAC com os parâmetros das condições dos aeroportos

In [48]:
vra_full = pd.merge_asof(
    vra.sort_values('Horário de Referência para Partida', ascending=True),
    metar.rename(columns={'aero': 'Sigla ICAO Aeroporto Origem'}).sort_values('hora', ascending=True),
    left_on='Horário de Referência para Partida',
    right_on='hora',
    by='Sigla ICAO Aeroporto Origem',
    direction='nearest'
)

# Change suffix new columns names in vra on merged_df to previsto
# example temperature -> temperature_previsto
vra_full.columns = [f'{col}_orig' if col in metar.columns else col for col in vra_full.columns]




vra_full = pd.merge_asof(
    vra_full.sort_values('Horário de Referência para Chegada', ascending=True),
    metar.rename(columns={'aero': 'Sigla ICAO Aeroporto Destino'}).sort_values('hora', ascending=True),
    left_on='Horário de Referência para Chegada',
    right_on='hora',
    by='Sigla ICAO Aeroporto Destino',
    direction='nearest'
)

vra_full.columns = [f'{col}_dest' if col in metar.columns else col for col in vra_full.columns]


vra_full.head()

,Sigla ICAO Empresa Aérea,Empresa Aérea,Número Voo,Código DI,Código Tipo Linha,Modelo Equipamento,Número de Assentos,Sigla ICAO Aeroporto Origem,Descrição Aeroporto Origem,Partida Prevista,...,metar_translated_dest,station_dest,time_dest,wind_direction_dest,wind_speed_dest,temperature_dest,dew_point_dest,visibility_dest,pressure_dest,sky_conditions_dest
0,GLO,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,2028,2,N,B38M,186,SBGL,AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GAL...,NaT,...,"{'station': 'SBRF', 'time': datetime.datetime(...",SBRF,2024-10-01 00:00:00,260.0,2.0,25.0,24.0,10000.0,1012.0,a few clouds at 1300 feet; broken clouds at 19...
1,SID,SIDERAL LINHAS AÉREAS LTDA.,9412,0,C,B73Y,0,SBCF,TANCREDO NEVES - CONFINS - MG - BRASIL,2022-06-01 00:40:00,...,"{'station': 'SBGR', 'time': datetime.datetime(...",SBGR,2024-10-01 02:00:00,100.0,3.0,16.0,15.0,10000.0,1019.0,scattered clouds at 800 feet; overcast at 1700...
2,SID,SIDERAL LINHAS AÉREAS LTDA.,9306,7,N,B73Y,0,SBBR,PRESIDENTE JUSCELINO KUBITSCHEK - BRASÍLIA - D...,2022-06-01 00:15:00,...,"{'station': 'SBGR', 'time': datetime.datetime(...",SBGR,2024-10-01 02:00:00,100.0,3.0,16.0,15.0,10000.0,1019.0,scattered clouds at 800 feet; overcast at 1700...
3,TTL,TOTAL LINHAS AÉREAS S.A.,5683,0,C,B722,0,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,2022-06-01 02:00:00,...,"{'station': 'SBGL', 'time': datetime.datetime(...",SBGL,2024-10-01 03:00:00,120.0,2.0,22.0,20.0,10000.0,1015.0,scattered clouds at 1800 feet; overcast at 300...
4,SID,SIDERAL LINHAS AÉREAS LTDA.,9303,0,C,B734,0,SBGR,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...,2022-06-01 02:40:00,...,"{'station': 'SBGL', 'time': datetime.datetime(...",SBGL,2024-10-01 04:00:00,100.0,2.0,22.0,20.0,10000.0,1015.0,scattered clouds at 1800 feet; overcast at 300...


Jutando informações de esperas em cabeceiras

In [49]:
vra_full = pd.merge_asof(
    vra_full.sort_values('Horário de Referência para Partida', ascending=True),
    esperas.rename(columns={'aero': 'Sigla ICAO Aeroporto Origem'}).sort_values('hora', ascending=True),
    left_on='Horário de Referência para Partida',
    right_on='hora',
    by='Sigla ICAO Aeroporto Origem',
    direction='nearest'
)

## change hora column name to hora_orig

vra_full.columns = [f'{col}_orig' if col in esperas.columns else col for col in vra_full.columns]

vra_full = pd.merge_asof(
    vra_full.sort_values('Horário de Referência para Chegada', ascending=True),
    esperas.rename(columns={'aero': 'Sigla ICAO Aeroporto Destino'}).sort_values('hora', ascending=True),
    left_on='Horário de Referência para Chegada',
    right_on='hora',
    by='Sigla ICAO Aeroporto Destino',
    direction='nearest'
)

## change hora column name to hora_dest

vra_full.columns = [f'{col}_dest' if col in esperas.columns else col for col in vra_full.columns]

In [51]:
# drop columns hora, time_prev and time_dest
vra_full.drop(columns=['time_orig', 'time_dest'], inplace=True)

In [53]:
vra_full[vra_full['esperas_orig'] > 10 ].reset_index(drop=True).loc[8]

Sigla ICAO Empresa Aérea                                                            GLO
Empresa Aérea                         GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...
Número Voo                                                                         1324
Código DI                                                                             0
Código Tipo Linha                                                                     N
Modelo Equipamento                                                                 B38M
Número de Assentos                                                                  186
Sigla ICAO Aeroporto Origem                                                        SBGR
Descrição Aeroporto Origem            GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO - ...
Partida Prevista                                                    2022-06-08 15:45:00
Partida Real                                                        2022-06-08 15:58:00
Sigla ICAO Aeroporto Destino    